In [2]:
# Rag Neo4j

In [20]:
from dotenv import load_dotenv
load_dotenv()

True

In [24]:
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
import os
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import TokenTextSplitter

In [22]:
from langchain_groq import ChatGroq

In [13]:
graph = Neo4jGraph(url="bolt://localhost:7687", username="neo4j", password="India@#2024")

### Lamma3

In [14]:
llm3 = ChatGroq(
    model="llama3-groq-70b-8192-tool-use-preview",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

#### GPT 3.5

In [15]:
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125")

In [ ]:
# llm_transformer = LLMGraphTransformer(llm=llm,
# allowed_nodes=["Person", "Country", "Organization"],
# allowed_relationships=["NATIONALITY", "LOCATED_IN", "DESIGNATION", "SPOUSE"])

In [16]:
llm_transformer = LLMGraphTransformer(llm=llm)

In [25]:
file = PyPDFLoader("Budget_Speech.pdf")
docs = file.load()
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(docs)
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [28]:
print(f"Nodes:{graph_documents[1].nodes}")
print(f"Relationships:{graph_documents[1].relationships}")

Nodes:[Node(id='Garib', type='Caste'), Node(id='Mahilayen', type='Caste'), Node(id='Yuva', type='Caste'), Node(id='Annadata', type='Caste'), Node(id='Pradhan Mantri Garib Kalyan Anna Yojana', type='Scheme'), Node(id='Msmes', type='Sector'), Node(id='Middle Class', type='Social group'), Node(id='Viksit Bharat', type='Strategy'), Node(id='Agriculture', type='Sector'), Node(id='Employment', type='Priority'), Node(id='Skilling', type='Priority'), Node(id='Human Resource Development', type='Priority'), Node(id='Social Justice', type='Priority'), Node(id='Manufacturing', type='Priority'), Node(id='Services', type='Priority'), Node(id='Urban Development', type='Priority'), Node(id='Energy Security', type='Priority'), Node(id='Infrastructure', type='Priority')]
Relationships:[Relationship(source=Node(id='Annadata', type='Caste'), target=Node(id='Pradhan Mantri Garib Kalyan Anna Yojana', type='Scheme'), type='SUPPORTS'), Relationship(source=Node(id='Msmes', type='Sector'), target=Node(id='Yuva'

In [29]:
graph.add_graph_documents(graph_documents, baseEntityLabel=True, include_source=True)
graph.refresh_schema()

In [30]:
print(graph.schema)

Node properties:
Person {id: STRING}
Country {id: STRING}
Organization {id: STRING}
Event {id: STRING}
Document {text: STRING, id: STRING, source: STRING, page: INTEGER}
Concept {id: STRING}
Position {id: STRING}
Date {id: STRING}
Group {id: STRING}
Caste {id: STRING}
Scheme {id: STRING}
Sector {id: STRING}
Social group {id: STRING}
Strategy {id: STRING}
Priority {id: STRING}
Framework {id: STRING}
Research setup {id: STRING}
Varieties {id: STRING}
Farming {id: STRING}
Farmers {id: STRING}
Process {id: STRING}
Missions {id: STRING}
Relationship properties:

The relationships:
(:Person)-[:FACING]->(:Concept)
(:Person)-[:BORN_AFTER]->(:Event)
(:Person)-[:IMPACTED]->(:Concept)
(:Person)-[:PRECEDED]->(:Concept)
(:Person)-[:REACHING]->(:Concept)
(:Person)-[:MENTIONED]->(:Group)
(:Person)-[:HOLDS_POSITION]->(:Position)
(:Person)-[:GAVE_SPEECH_ON]->(:Date)
(:Person)-[:ADDRESSED]->(:Person)
(:Organization)-[:ESTIMATES_POPULATION]->(:Person)
(:Document)-[:MENTIONS]->(:Organization)
(:Document)-

#### Query Graph DB

In [31]:
from langchain.chains import GraphCypherQAChain

In [ ]:
# Prompt

In [62]:
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], 
    template=CYPHER_GENERATION_TEMPLATE
)

In [32]:
chain = GraphCypherQAChain.from_llm(
    llm, graph=graph, verbose=True
)

In [52]:
chain.invoke({"query": "how many New Varieties are introduced?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (v:Varieties) RETURN COUNT(v)
Full Context:
[{'COUNT(v)': 2}]

> Finished chain.


{'query': 'how many New Varieties are introduced?',
 'result': 'Based on the information provided, 2 new varieties are introduced.'}